<a href="https://colab.research.google.com/github/heriswn/LatihanDTS/blob/master/2507_LDAdanPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TUGAS PCA DAN LDA

## Exercise 1

### Prepare

In [0]:
import pandas as pd
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer

### Menginput dataset Breast Cancer

In [0]:
bc = load_breast_cancer()
data = bc.data
labels = np.reshape(bc.target,(569,1))
features = bc.feature_names
features_labels = np.append(features,'label')
final_data = np.concatenate([data,labels],axis=1)
df = pd.DataFrame(final_data)
df.columns = features_labels
df['label'].replace(0, 'Malignant',inplace=True)
df['label'].replace(1, 'Benign',inplace=True)

In [50]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,label
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,Malignant
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,Malignant
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,Malignant
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,Malignant
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,Malignant


### Cek missing values

In [0]:
cols=['mean radius', 'mean texture', 'mean perimeter', 'mean area',
 'mean smoothness', 'mean compactness', 'mean concavity',
 'mean concave points', 'mean symmetry', 'mean fractal dimension',
 'radius error', 'texture error', 'perimeter error', 'area error',
 'smoothness error', 'compactness error', 'concavity error',
 'concave points error', 'symmetry error', 'fractal dimension error',
 'worst radius', 'worst texture', 'worst perimeter', 'worst area',
 'worst smoothness', 'worst compactness', 'worst concavity',
 'worst concave points', 'worst symmetry', 'worst fractal dimension', 'label']

In [52]:
for i in cols:
    nr=df[df[i]=='?'].shape[0]
    if nr>0:
        print(i,':',nr)
    else:
        print(i,': No missing values')

mean radius : No missing values
mean texture : No missing values
mean perimeter : No missing values
mean area : No missing values
mean smoothness : No missing values
mean compactness : No missing values
mean concavity : No missing values
mean concave points : No missing values
mean symmetry : No missing values
mean fractal dimension : No missing values
radius error : No missing values
texture error : No missing values
perimeter error : No missing values
area error : No missing values
smoothness error : No missing values
compactness error : No missing values
concavity error : No missing values
concave points error : No missing values
symmetry error : No missing values
fractal dimension error : No missing values
worst radius : No missing values
worst texture : No missing values
worst perimeter : No missing values
worst area : No missing values
worst smoothness : No missing values
worst compactness : No missing values
worst concavity : No missing values
worst concave points : No missing v

/usr/local/lib/python3.6/dist-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


## Klasifikasi

### Menginput model yang akan diujikan dan menginput subset random training dan testing

In [0]:
models = {
"knn": KNeighborsClassifier(n_neighbors=1),
"naive_bayes": GaussianNB(),
"logit": LogisticRegression(solver="lbfgs", multi_class="auto"),
"svm": SVC(kernel="rbf", gamma="auto"),
"decision_tree": DecisionTreeClassifier(),
"random_forest": RandomForestClassifier(n_estimators=100),
}

### Hasil dengan model Logistic Regression dr dataset awal

In [54]:
labels = df.columns[:30]

X = df.loc[:, features].values
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

LR = LogisticRegression(random_state=None, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
LR.predict(X_test)

print(round(LR.score(X_test, y_test), 4))
print(classification_report(y_test, LR.predict(X_test)))

0.9357
              precision    recall  f1-score   support

      Benign       0.94      0.96      0.95       111
   Malignant       0.93      0.88      0.91        60

    accuracy                           0.94       171
   macro avg       0.93      0.92      0.93       171
weighted avg       0.94      0.94      0.94       171



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [55]:
RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=None).fit(X_train, y_train)
RF.predict(X_test)
print(round(RF.score(X_test, y_test), 4))
print(classification_report(y_test, RF.predict(X_test)))

0.9766
              precision    recall  f1-score   support

      Benign       0.97      0.99      0.98       111
   Malignant       0.98      0.95      0.97        60

    accuracy                           0.98       171
   macro avg       0.98      0.97      0.97       171
weighted avg       0.98      0.98      0.98       171



In [56]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
x = df.loc[:, features].values
# normalizing: mean=0; std=1
x = StandardScaler().fit_transform(x)
pca = PCA(n_components=3)
pC_df = pca.fit_transform(x)
principal_df = pd.DataFrame(data = pC_df, columns = ['PC1', 'PC2', 'PC3'])
principal_df.tail()

,PC1,PC2,PC3
564,6.439315,-3.576817,2.459483
565,3.793382,-3.584048,2.088476
566,1.256179,-1.902297,0.562733
567,10.374794,1.672010,-1.877026
568,-5.475243,-0.670636,1.490432


In [57]:
print('Explained variation per principal component:{}'.format(pca.explained_variance_ratio_))

Explained variation per principal component:[0.44272026 0.18971182 0.09393163]


In [58]:
labels = df.columns[:30]


X = principal_df
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=None)

LR = LogisticRegression(random_state=None, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)
LR.predict(X_test)

print(round(LR.score(X_test, y_test), 4))
print(classification_report(y_test, LR.predict(X_test)))

0.9591
              precision    recall  f1-score   support

      Benign       0.97      0.96      0.97       103
   Malignant       0.94      0.96      0.95        68

    accuracy                           0.96       171
   macro avg       0.96      0.96      0.96       171
weighted avg       0.96      0.96      0.96       171



In [59]:
RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=None).fit(X_train, y_train)
RF.predict(X_test)
print(round(RF.score(X_test, y_test), 4))
print(classification_report(y_test, RF.predict(X_test)))

0.9591
              precision    recall  f1-score   support

      Benign       0.94      0.99      0.97       103
   Malignant       0.98      0.91      0.95        68

    accuracy                           0.96       171
   macro avg       0.96      0.95      0.96       171
weighted avg       0.96      0.96      0.96       171

